In [59]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
from calendar import monthrange

In [60]:
def get_link(month : int, year : int, city='Kursk') -> str:
    if city == 'Kursk':
        city_id = 34009 #Адресс Места Роста Сои в Курск
    elif city == 'Voronez':
        city_id = 34238 #Адресс Места Роста Сои в Воронеже 
    day_start, day_end = monthrange(year, month)
    
    return "http://pogodaiklimat.ru/weather.php?id=" + f"{city_id}" + "&bday=1" + "&fday=" + f"{day_end}" + "&amonth=" + f"{month}" + "&ayear=" + f"{year}" + "&bot=2"

In [61]:
def get_pred_single(month, year, city='Kursk'):
    if city == 'Kursk':
        page = requests.get(get_link(month, year, city='Kursk'))
    elif city == 'Voronez':
        page = requests.get(get_link(month, year, city='Voronez'))        

    page.encoding = 'utf-8'
    html = page.text

    soup = bs(html, 'html.parser')

    data_1 = []

    rows = soup.table.find_all('tr')
    for row in rows:
        columns = row.find_all('td')
        data_1.append([column.text for column in columns])

    data_table = pd.DataFrame(np.array(data_1[1:]).astype(float), columns=data_1[0][0].replace(',', '').split('\t'))

    data_2 = []

    rows = soup.find_all('tr')

    for row in rows[len(data_1):]:
        columns = row.find_all('td')
        data_2.append([column.text for column in columns])

    headers = [0] * 18 
    headers[:1] = data_2[0][0].split()
    headers[2:] = data_2[0][1:]

    weather_table =  pd.DataFrame(np.array(data_2[1:]), columns=headers)

    return pd.merge(data_table, weather_table, left_index=True, right_index=True)

In [62]:
kursks_2017 = []

for month in range(5, 10):
    kursks_2017.append(get_pred_single(month, 2017, 'Voronez'))

In [63]:
pd.concat(kursks_2017).to_csv('2017.csv')

In [64]:
data = pd.read_csv('2017.csv')
data 

,Unnamed: 0,Время (UTC),дата,Ветер,"(напр.,м/с)",Видим.,Явления,Облачность,Т(С),Тd(С),...,Тe(С),Тes(С),Комфортность,P(гПа),Po(гПа),Тmin(С),Tmax(С),R(мм),R24(мм),S(см)
0,0,0.0,1.05,ЮВ,2,10 км,NaN,8/0[Ci sp],13.8,1.9,...,14.0,14.0,NaN,1023.4,1004.8,NaN,NaN,NaN,NaN,NaN
1,1,3.0,1.05,ЮВ,3,10 км,NaN,8/0[Ac trans Ci sp],12.9,3.3,...,12.0,12.0,NaN,1023.3,1004.7,12.4,NaN,NaN,NaN,NaN
2,2,6.0,1.05,В,3,10 км,NaN,9/0[Ci fib],16.9,2.7,...,17.0,22.0,NaN,1023.4,1005.0,NaN,NaN,NaN,NaN,NaN
3,3,9.0,1.05,ЮВ,5 {12},10 км,NaN,8/0[Ac trans Ci fib],24.0,-0.8,...,23.0,28.0,комфорт,1022.1,1004.1,NaN,NaN,NaN,NaN,NaN
4,4,12.0,1.05,ЮВ,6-11 {13},10 км,NaN,10/3 600 м[Cb calv As trans],24.6,-1.5,...,23.0,24.0,комфорт,1020.9,1003.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,235,9.0,30.09,С,2,10 км,NaN,10/8 300 м[Frnb As op],8.9,4.6,...,7.0,8.0,NaN,1026.1,1007.1,NaN,NaN,NaN,NaN,NaN
1220,236,12.0,30.09,СЗ,1,10 км,в посл. час ливневой дождь,10/10 600 м[Cb calv],9.0,6.9,...,8.0,8.0,NaN,1025.3,1006.4,NaN,NaN,NaN,NaN,NaN
1221,237,15.0,30.09,С,2,10 км,{ливн. осадки},10/10 600 м[Cb calv],8.6,6.2,...,7.0,7.0,NaN,1025.6,1006.6,NaN,9.7,1.0,NaN,NaN
1222,238,18.0,30.09,С,2,10 км,{ливн. осадки},3/3 600 м[Cb calv],6.8,4.3,...,5.0,5.0,NaN,1026.2,1007.1,NaN,NaN,NaN,NaN,NaN
